In [21]:
import pandas as pd
import re
import string
def generate_sequence():
    letters = string.ascii_uppercase
    single_letter_sequences = list(letters) # Generate single letter sequences
    double_letter_sequences = [a + b for a in letters for b in letters] # Generate double letter sequences
    triple_letter_sequences = [a + b + c for a in letters for b in letters for c in letters] # Generate triple letter sequences
    all_sequences = single_letter_sequences + double_letter_sequences + triple_letter_sequences # Combine all sequences
    return all_sequences

def number_id_naming(name):
    number_id = int(re.search(r"_(\d+)_", name).group(1))  # extract number id from the element
    number_id += 1  # Because the first element is 0, and it should be 1
    number_id = str(number_id)[::-1].zfill(len(str(number_id)) + 1)[::-1]  # fill the number with 0
    # number_id = int(number_id)
    return number_id

def subfamily_naming(chromosome, naming, sequences):
    abc_seq = generate_sequence()  # it will generate from A to ZZZ
    letter_slider = 0
    named_elements = []
    # subfamily_abc_counter_id = "A"
    for _, seqs in enumerate(sequences):  # selecting list
        if len(seqs) > 1:
            subfamily = []
            for _, element in enumerate(seqs):  # selecting element inside a list.
                chr_id = re.search(r"\d+", chromosome).group(0)
                number_id = number_id_naming(element)
                subfamily.append(f"{naming}_c{chr_id}.{number_id}{abc_seq[letter_slider]}")
            letter_slider += 1
            named_elements.append(subfamily)
        else: # if len(seqs) == 1
            chr_id = re.search(r"\d+", chromosome).group(0)
            number_id = number_id_naming(seqs[0])
            named_elements.append([f"{naming}_c{chr_id}.{number_id}"])

    return named_elements

In [2]:
path_file = "~/Documents/Work_CBMSO/Leish_Data_Analysis/21.Subfamilies_classification/subfamilies_classification/LinJ.01/families_LinJ.01.csv"

In [26]:
data_main = [['Seq_13_LinJ.02'], ['Seq_14_LinJ.02', 'Seq_15_LinJ.02', 'Seq_17_LinJ.02', 'Seq_21_LinJ.02'], ['Seq_16_LinJ.02', 'Seq_20_LinJ.02', 'Seq_22_LinJ.02'], ['Seq_18_LinJ.02', 'Seq_23_LinJ.02', 'Seq_24_LinJ.02'], ['Seq_19_LinJ.02'], ['Seq_25_LinJ.02'], ['Seq_26_LinJ.02'], ['Seq_27_LinJ.02', 'Seq_29_LinJ.02'], ['Seq_28_LinJ.02']]
[print(elem) for elem in data_main]

['Seq_13_LinJ.02']
['Seq_14_LinJ.02', 'Seq_15_LinJ.02', 'Seq_17_LinJ.02', 'Seq_21_LinJ.02']
['Seq_16_LinJ.02', 'Seq_20_LinJ.02', 'Seq_22_LinJ.02']
['Seq_18_LinJ.02', 'Seq_23_LinJ.02', 'Seq_24_LinJ.02']
['Seq_19_LinJ.02']
['Seq_25_LinJ.02']
['Seq_26_LinJ.02']
['Seq_27_LinJ.02', 'Seq_29_LinJ.02']
['Seq_28_LinJ.02']


[None, None, None, None, None, None, None, None, None]

May I do a dict to get the correct order? and then index the dict to replace the elements?

In [ ]:
test = 'Seq_13_LinJ.02'


In [27]:
new_list = [name for element in data_main for name in element]
print(new_list)

['Seq_13_LinJ.02', 'Seq_14_LinJ.02', 'Seq_15_LinJ.02', 'Seq_17_LinJ.02', 'Seq_21_LinJ.02', 'Seq_16_LinJ.02', 'Seq_20_LinJ.02', 'Seq_22_LinJ.02', 'Seq_18_LinJ.02', 'Seq_23_LinJ.02', 'Seq_24_LinJ.02', 'Seq_19_LinJ.02', 'Seq_25_LinJ.02', 'Seq_26_LinJ.02', 'Seq_27_LinJ.02', 'Seq_29_LinJ.02', 'Seq_28_LinJ.02']


In [28]:
new_list = sorted(new_list, key=lambda x: int(re.search(r'_(\d+)_', x).group(1)))
print(new_list)

['Seq_13_LinJ.02', 'Seq_14_LinJ.02', 'Seq_15_LinJ.02', 'Seq_16_LinJ.02', 'Seq_17_LinJ.02', 'Seq_18_LinJ.02', 'Seq_19_LinJ.02', 'Seq_20_LinJ.02', 'Seq_21_LinJ.02', 'Seq_22_LinJ.02', 'Seq_23_LinJ.02', 'Seq_24_LinJ.02', 'Seq_25_LinJ.02', 'Seq_26_LinJ.02', 'Seq_27_LinJ.02', 'Seq_28_LinJ.02', 'Seq_29_LinJ.02']


In [29]:
counter = 1
main_dict = {}
for name_id in new_list:
    main_dict[name_id] = counter
    counter += 1

In [31]:
for key, value in main_dict.items(): print(key, value)

Seq_13_LinJ.02 1
Seq_14_LinJ.02 2
Seq_15_LinJ.02 3
Seq_16_LinJ.02 4
Seq_17_LinJ.02 5
Seq_18_LinJ.02 6
Seq_19_LinJ.02 7
Seq_20_LinJ.02 8
Seq_21_LinJ.02 9
Seq_22_LinJ.02 10
Seq_23_LinJ.02 11
Seq_24_LinJ.02 12
Seq_25_LinJ.02 13
Seq_26_LinJ.02 14
Seq_27_LinJ.02 15
Seq_28_LinJ.02 16
Seq_29_LinJ.02 17


In [22]:
subfamily_naming("LinJ.01", "LinJ", data_main)

[['LinJ_c01.10'],
 ['LinJ_c01.20A',
  'LinJ_c01.30A',
  'LinJ_c01.50A',
  'LinJ_c01.60A',
  'LinJ_c01.90A',
  'LinJ_c01.100A',
  'LinJ_c01.120A'],
 ['LinJ_c01.40'],
 ['LinJ_c01.70'],
 ['LinJ_c01.80'],
 ['LinJ_c01.110'],
 ['LinJ_c01.130']]